In [0]:
storageAccountKey = 'account_access_key'
spark.conf.set("account_link",storageAccountKey)

_Task_ 1: Data Ingestion

Q1. Fetch the US 2025 public holidays from the provided API using Python.
Q2. Convert the JSON response into a tabular format using Pandas.
Q3. Save the dataset as a CSV file (us_public_holidays_2025.csv).

In [0]:
import pandas as pd
import requests

In [0]:
url="https://date.nager.at/api/v3/PublicHolidays/2025/US"
response=requests.get(url)
data= response.json()
df = pd.DataFrame(data)
display(df)

date,localName,name,countryCode,fixed,global,counties,launchYear,types
2025-01-01,New Year's Day,New Year's Day,US,false,true,null,null,List(Public)
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,false,true,null,null,List(Public)
2025-02-12,Lincoln's Birthday,Lincoln's Birthday,US,false,false,"List(US-CA, US-CT, US-IL, US-IN, US-KY, US-MI, US-NY, US-MO, US-OH)",null,List(Observance)
2025-02-17,Washington's Birthday,Presidents Day,US,false,true,null,null,List(Public)
2025-04-18,Good Friday,Good Friday,US,false,false,"List(US-CT, US-DE, US-HI, US-IN, US-KY, US-LA, US-NC, US-ND, US-NJ, US-TN)",null,List(Public)
2025-04-18,Good Friday,Good Friday,US,false,false,List(US-TX),null,List(Optional)
2025-05-08,Truman Day,Truman Day,US,false,false,List(US-MO),null,"List(School, Authorities)"
2025-05-26,Memorial Day,Memorial Day,US,false,true,null,null,List(Public)
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,US,false,true,null,null,List(Public)
2025-07-04,Independence Day,Independence Day,US,false,true,null,null,List(Public)


In [0]:
df.to_csv("us_public_holidays_2025.csv", index=False)
print("Created csv successfully")

Created csv successfully


In [0]:
df.columns.values
df.dtypes

date           object
localName      object
name           object
countryCode    object
fixed            bool
global           bool
counties       object
launchYear     object
types          object
dtype: object

Task 2: Load & Explore in PySpark

Q4. Load the generated CSV into a PySpark DataFrame. 
Q5. Print the schema and identify which columns might need data type conversion. 
Q6. Display the top 5 rows for validation.

In [0]:
sourceSparkDF = spark.createDataFrame(df)
display(sourceSparkDF)

date,localName,name,countryCode,fixed,global,counties,launchYear,types
2025-01-01,New Year's Day,New Year's Day,US,false,true,null,null,List(Public)
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,false,true,null,null,List(Public)
2025-02-12,Lincoln's Birthday,Lincoln's Birthday,US,false,false,"List(US-CA, US-CT, US-IL, US-IN, US-KY, US-MI, US-NY, US-MO, US-OH)",null,List(Observance)
2025-02-17,Washington's Birthday,Presidents Day,US,false,true,null,null,List(Public)
2025-04-18,Good Friday,Good Friday,US,false,false,"List(US-CT, US-DE, US-HI, US-IN, US-KY, US-LA, US-NC, US-ND, US-NJ, US-TN)",null,List(Public)
2025-04-18,Good Friday,Good Friday,US,false,false,List(US-TX),null,List(Optional)
2025-05-08,Truman Day,Truman Day,US,false,false,List(US-MO),null,"List(School, Authorities)"
2025-05-26,Memorial Day,Memorial Day,US,false,true,null,null,List(Public)
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,US,false,true,null,null,List(Public)
2025-07-04,Independence Day,Independence Day,US,false,true,null,null,List(Public)


In [0]:
sourceCSVFilePath = "abfss://working-labs@myfirstdemostorageacc.dfs.core.windows.net/bronze"

In [0]:
from pyspark.sql.functions import col, concat_ws

sourceSparkDF = sourceSparkDF.withColumn("counties", concat_ws(",", col("counties")))
sourceSparkDF.write.mode("overwrite").save(sourceCSVFilePath)


In [0]:
sourceSparkDF.printSchema()

root
 |-- date: string (nullable = true)
 |-- localName: string (nullable = true)
 |-- name: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- fixed: boolean (nullable = true)
 |-- global: boolean (nullable = true)
 |-- counties: string (nullable = false)
 |-- launchYear: void (nullable = true)
 |-- types: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
sourceSparkDF.show(5)
display(sourceSparkDF)

+----------+--------------------+--------------------+-----------+-----+------+--------------------+----------+------------+
|      date|           localName|                name|countryCode|fixed|global|            counties|launchYear|       types|
+----------+--------------------+--------------------+-----------+-----+------+--------------------+----------+------------+
|2025-01-01|      New Year's Day|      New Year's Day|         US|false|  true|                    |      NULL|    [Public]|
|2025-01-20|Martin Luther Kin...|Martin Luther Kin...|         US|false|  true|                    |      NULL|    [Public]|
|2025-02-12|  Lincoln's Birthday|  Lincoln's Birthday|         US|false| false|US-CA,US-CT,US-IL...|      NULL|[Observance]|
|2025-02-17|Washington's Birt...|      Presidents Day|         US|false|  true|                    |      NULL|    [Public]|
|2025-04-18|         Good Friday|         Good Friday|         US|false| false|US-CT,US-DE,US-HI...|      NULL|    [Public]|


date,localName,name,countryCode,fixed,global,counties,launchYear,types
2025-01-01,New Year's Day,New Year's Day,US,false,true,,null,List(Public)
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,false,true,,null,List(Public)
2025-02-12,Lincoln's Birthday,Lincoln's Birthday,US,false,false,"US-CA,US-CT,US-IL,US-IN,US-KY,US-MI,US-NY,US-MO,US-OH",null,List(Observance)
2025-02-17,Washington's Birthday,Presidents Day,US,false,true,,null,List(Public)
2025-04-18,Good Friday,Good Friday,US,false,false,"US-CT,US-DE,US-HI,US-IN,US-KY,US-LA,US-NC,US-ND,US-NJ,US-TN",null,List(Public)
2025-04-18,Good Friday,Good Friday,US,false,false,US-TX,null,List(Optional)
2025-05-08,Truman Day,Truman Day,US,false,false,US-MO,null,"List(School, Authorities)"
2025-05-26,Memorial Day,Memorial Day,US,false,true,,null,List(Public)
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,US,false,true,,null,List(Public)
2025-07-04,Independence Day,Independence Day,US,false,true,,null,List(Public)


Task 3: Data Type & Date Transformations

Q7. Convert the date string column to a proper DateType.
Q8. Add two new columns:

holiday_month (Extract month from the date)
weekday_name (Extract full weekday name, e.g., Monday)
Q9. Which day of the week has the most holidays in 2025?


In [0]:
from pyspark.sql.functions import to_date
sourceSparkDF = sourceSparkDF.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
sourceSparkDF.printSchema()

root
 |-- date: date (nullable = true)
 |-- localName: string (nullable = true)
 |-- name: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- fixed: boolean (nullable = true)
 |-- global: boolean (nullable = true)
 |-- counties: string (nullable = false)
 |-- launchYear: void (nullable = true)
 |-- types: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
from pyspark.sql.functions import monthname, date_format
sourceSparkDF = sourceSparkDF.withColumn("holiday_month", monthname('date'))
sourceSparkDF = sourceSparkDF.withColumn("Weekday_name", date_format('date', 'EEEE'))
display(sourceSparkDF)

date,localName,name,countryCode,fixed,global,counties,launchYear,types,holiday_month,Weekday_name
2025-01-01,New Year's Day,New Year's Day,US,false,true,,null,List(Public),Jan,Wednesday
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,false,true,,null,List(Public),Jan,Monday
2025-02-12,Lincoln's Birthday,Lincoln's Birthday,US,false,false,"US-CA,US-CT,US-IL,US-IN,US-KY,US-MI,US-NY,US-MO,US-OH",null,List(Observance),Feb,Wednesday
2025-02-17,Washington's Birthday,Presidents Day,US,false,true,,null,List(Public),Feb,Monday
2025-04-18,Good Friday,Good Friday,US,false,false,"US-CT,US-DE,US-HI,US-IN,US-KY,US-LA,US-NC,US-ND,US-NJ,US-TN",null,List(Public),Apr,Friday
2025-04-18,Good Friday,Good Friday,US,false,false,US-TX,null,List(Optional),Apr,Friday
2025-05-08,Truman Day,Truman Day,US,false,false,US-MO,null,"List(School, Authorities)",May,Thursday
2025-05-26,Memorial Day,Memorial Day,US,false,true,,null,List(Public),May,Monday
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,US,false,true,,null,List(Public),Jun,Thursday
2025-07-04,Independence Day,Independence Day,US,false,true,,null,List(Public),Jul,Friday


In [0]:
from pyspark.sql.functions import col, count, desc
weekwithmostholidaysDF = (
    sourceSparkDF
    .groupBy("Weekday_name")
    .agg(count("*").alias("count"))
    .orderBy(col("count").desc())
    .limit(1)
)
display(weekwithmostholidaysDF) # Monday

Weekday_name,count
Monday,6


Task 4: Business Use Case - Holiday Analysis

Q10. Group holidays by weekday_name and count them.
Q11. Suggest 2 insights HR can use based on weekday-wise distribution.
Q12. Identify months with more than 2 holidays. What are they?

In [0]:
weekdayholidaysDF = (
  sourceSparkDF
  .groupBy("Weekday_name")
  .agg(count("*").alias("count")
).show()
)

+------------+-----+
|Weekday_name|count|
+------------+-----+
|   Wednesday|    2|
|      Monday|    6|
|      Friday|    3|
|    Thursday|    4|
|     Tuesday|    1|
+------------+-----+



In [0]:
monthwithmoreholidaysDF = (
    sourceSparkDF
    .groupBy("holiday_month")
    .agg(count("*").alias("count"))
    .filter("count > 2")
).show()

+-------------+-----+
|holiday_month|count|
+-------------+-----+
+-------------+-----+



Task 5: Data Storage & Delivery

Q13. Save the transformed data in:
JSON format
Parquet format
Ensure overwrite mode is enabled.
Q14. Mount or use the Azure Data Lake Storage (ADLS) path to write output files.
Q15. Document the full ADLS path used and the type of container.

In [0]:
from pyspark.sql.functions import lit, col, concat_ws
sourceCSVFilePath = "abfss://working-labs@myfirstdemostorageacc.dfs.core.windows.net/holidays2025.csv"
sourceSparkDF = sourceSparkDF.withColumn("launchYear",lit(None).cast("int"))
sourceSparkDF = sourceSparkDF.withColumn("types", concat_ws(",", col("types")))
sourceSparkDF.write\
.mode("overwrite")\
.format("csv")\
.option("headers", True)\
.save(sourceCSVFilePath)


In [0]:
sourceCSVFilePath = "abfss://working-labs@myfirstdemostorageacc.dfs.core.windows.net/holidays2025.csv"
targetJsonFilePath = "abfss://working-labs@myfirstdemostorageacc.dfs.core.windows.net/PublicHolidays2025.json"

In [0]:
sourceCSVFileDF = (
    spark.
    read.
    option("header", "true").
    csv(
        sourceCSVFilePath,
)
)

In [0]:
df.dtypes

date           object
localName      object
name           object
countryCode    object
fixed            bool
global           bool
counties       object
launchYear     object
types          object
dtype: object

In [0]:
from pyspark.sql.types import *
sourceCSVFileSchema=StructType([
    StructField("date", StringType(), True),
    StructField("localName ",  StringType(), True),
    StructField("name", StringType(), True),
    StructField("countryCode ", IntegerType(), True) ,
    StructField("fixed", BooleanType(), True),
    StructField("global", BooleanType(), True),
    StructField("counties", StringType(), True),
    StructField("launchYear", StringType(), True),
    StructField("types ", DecimalType(10,2),True)
])
sourceCSVFileDF=spark.read.schema(sourceCSVFileSchema).csv(sourceCSVFilePath)
sourceCSVFileDF.write.mode("overwrite")

In [0]:
sourceCSVFileDF=(
    spark.
    read.
    schema(sourceCSVFileSchema).
    csv(sourceCSVFilePath)
)

In [0]:
display(sourceCSVFileDF)

date,localName,name,countryCode,fixed,global,counties,launchYear,types
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,null,false,true,null,null,null
2025-07-04,Independence Day,Independence Day,null,false,true,null,null,null
2025-09-01,Labour Day,Labor Day,null,false,true,null,null,null
2025-10-13,Columbus Day,Columbus Day,null,false,false,"US-AL,US-AZ,US-CO,US-CT,US-GA,US-ID,US-IL,US-IN,US-IA,US-KS,US-KY,US-LA,US-ME,US-MD,US-MA,US-MS,US-MO,US-MT,US-NE,US-NH,US-NJ,US-NM,US-NY,US-NC,US-OH,US-OK,US-PA,US-RI,US-SC,US-TN,US-UT,US-VA,US-WV",null,null
2025-10-13,Indigenous Peoples' Day,Indigenous Peoples' Day,null,false,false,"US-AK,US-AL,US-CA,US-HI,US-IA,US-LA,US-ME,US-MI,US-MN,US-NC,US-NE,US-NM,US-OK,US-OR,US-SD,US-TX,US-VA,US-VT,US-WI",null,null
2025-11-11,Veterans Day,Veterans Day,null,false,true,null,null,null
2025-11-27,Thanksgiving Day,Thanksgiving Day,null,false,true,null,null,null
2025-12-25,Christmas Day,Christmas Day,null,false,true,null,null,null
2025-01-01,New Year's Day,New Year's Day,null,false,true,null,null,null
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",null,false,true,null,null,null


In [0]:
sourceCSVFileDF.printSchema()

root
 |-- date: string (nullable = true)
 |-- localName : string (nullable = true)
 |-- name: string (nullable = true)
 |-- countryCode : integer (nullable = true)
 |-- fixed: boolean (nullable = true)
 |-- global: boolean (nullable = true)
 |-- counties: string (nullable = true)
 |-- launchYear: string (nullable = true)
 |-- types : decimal(10,2) (nullable = true)



In [0]:
(
    sourceCSVFileDF.
    write.
    mode("overwrite").
    json(targetJsonFilePath)
)

In [0]:
targetPARQUETFilePath='abfss://working-labs@myfirstdemostorageacc.dfs.core.windows.net/PublicHolidaysparquet'

In [0]:
from pyspark.sql.types import *
sourceCSVFileSchema1=StructType([
    StructField("date", StringType(), True),
    StructField("localName",  StringType(), True),
    StructField("name", StringType(), True),
    StructField("countryCode", IntegerType(), True) ,
    StructField("fixed", BooleanType(), True),
    StructField("global", BooleanType(), True),
    StructField("counties", StringType(), True),
    StructField("launchYear", StringType(), True),
    StructField("types", DecimalType(10,2),True)
])
sourceCSVFileDF=spark.read.schema(sourceCSVFileSchema).csv(sourceCSVFilePath)
sourceCSVFileDF.write.mode("overwrite")

In [0]:
sourceCSVFileDF = ( spark.
                   read.
                   schema(sourceCSVFileSchema1).
                   csv(sourceCSVFilePath) 
                  )

In [0]:
(
    sourceCSVFileDF.
    write.
    mode("overwrite").
    format("delta").
    save(targetPARQUETFilePath)
)

In [0]:
(
    spark.
    read.
    load(targetPARQUETFilePath).
    display()
)

date,localName,name,countryCode,fixed,global,counties,launchYear,types
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,null,false,true,null,null,null
2025-07-04,Independence Day,Independence Day,null,false,true,null,null,null
2025-09-01,Labour Day,Labor Day,null,false,true,null,null,null
2025-10-13,Columbus Day,Columbus Day,null,false,false,"US-AL,US-AZ,US-CO,US-CT,US-GA,US-ID,US-IL,US-IN,US-IA,US-KS,US-KY,US-LA,US-ME,US-MD,US-MA,US-MS,US-MO,US-MT,US-NE,US-NH,US-NJ,US-NM,US-NY,US-NC,US-OH,US-OK,US-PA,US-RI,US-SC,US-TN,US-UT,US-VA,US-WV",null,null
2025-10-13,Indigenous Peoples' Day,Indigenous Peoples' Day,null,false,false,"US-AK,US-AL,US-CA,US-HI,US-IA,US-LA,US-ME,US-MI,US-MN,US-NC,US-NE,US-NM,US-OK,US-OR,US-SD,US-TX,US-VA,US-VT,US-WI",null,null
2025-11-11,Veterans Day,Veterans Day,null,false,true,null,null,null
2025-11-27,Thanksgiving Day,Thanksgiving Day,null,false,true,null,null,null
2025-12-25,Christmas Day,Christmas Day,null,false,true,null,null,null
2025-01-01,New Year's Day,New Year's Day,null,false,true,null,null,null
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",null,false,true,null,null,null
